In [21]:
# %matplotlib widget

from __future__ import annotations

import re
from collections import defaultdict
from dataclasses import dataclass, field
from itertools import permutations, product
from math import inf
from random import choice

import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import numpy.typing as npt
from mpl_toolkits.mplot3d import axes3d
from numpy import int_, object_
from numpy.typing import NDArray
from test_utilities import run_tests_params
from util import print_hex

COLORS = list(mcolors.CSS4_COLORS.keys())

<link href="style.css" rel="stylesheet"></link>
...


In [22]:
from copy import deepcopy
from IntcodeComputer import IntcodeComputer

with open("../input/day21.txt") as f:
    puzzle = f.read()

# print(f"Part I: {frequency(puzzle)}")
program = """
NOT C J
AND D J
NOT A T
OR T J
WALK
"""


ic = IntcodeComputer(puzzle)
iter = deepcopy(ic).run()

out = iter.send(None)
print(f"{None if out is None else chr(out)}", end="")

while out != 10:
    out = iter.send(None)
    print(f"{None if out is None else chr(out)}", end="")

iter.send(None)

for line in program.strip().splitlines():
    for c in (ord(c) for c in line.strip()):
        out = iter.send(c)

    out = iter.send(10)


print(f"{None if out is None else chr(out)}", end=" ")
try:
    while True:
        out = iter.send(None)
        print(f"{None if out is None else chr(out)}", end="")
except (StopIteration, ValueError):
    print("Part I:", out)

Input instructions:

 Walking...

Part I: 19354818


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>19354818</code>.</p><p class="day-success">The first half of this puzzle is complete! It provides one gold star: *</p>
<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p>There are many areas the springdroid can't reach.  You flip through the manual and discover a way to <em>increase its sensor range</em>.</p>
<p>Instead of ending your springcode program with <code>WALK</code>, use <code>RUN</code>. Doing this will enable <em>extended sensor mode</em>, capable of sensing ground up to <em>nine tiles away</em>. This data is available in <em>five new read-only registers</em>:</p>
<ul>
<li>Register <code>E</code> indicates whether there is ground <em>five</em> tiles away.</li>
<li>Register <code>F</code> indicates whether there is ground <em>six</em> tiles away.</li>
<li>Register <code>G</code> indicates whether there is ground <em>seven</em> tiles away.</li>
<li>Register <code>H</code> indicates whether there is ground <em>eight</em> tiles away.</li>
<li>Register <code>I</code> indicates whether there is ground <em>nine</em> tiles away.</li>
</ul>
<p>All other functions remain the same.</p>
<p>Successfully survey the rest of the hull by ending your program with <code>RUN</code>.  <em>What amount of hull damage does the springdroid now report?</em></p>
</article>

</main>


In [23]:
program = """
NOT C J
AND D J
AND H J
NOT B T
AND D T
OR T J
NOT A T
OR T J
RUN
"""

ic = IntcodeComputer(puzzle)
iter = deepcopy(ic).run()

out = iter.send(None)
print(f"{None if out is None else chr(out)}", end="")

while out != 10:
    out = iter.send(None)
    print(f"{None if out is None else chr(out)}", end="")

iter.send(None)

for line in program.strip().splitlines():
    for c in (ord(c) for c in line.strip()):
        out = iter.send(c)

    out = iter.send(10)


print(f"{None if out is None else chr(out)}", end=" ")
try:
    while True:
        out = iter.send(None)
        print(f"{None if out is None else chr(out)}", end="")
except (StopIteration, ValueError):
    print("Part II:", out)

Input instructions:

 Running...

Part II: 1143787220


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>1143787220</code>.</p><p class="day-success">Both parts of this puzzle are complete! They provide two gold stars: **</p>

</main>
